In [3]:
# after we install all the packages, import all of them for the use in today's lecture!
import wrds as wrds
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [6]:
db = wrds.Connection(wrds_username='marzipan')


WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


In [7]:
db.create_pgpass_file()

In [8]:
params = {}
params['secids'] = ('108105',)

sql = '''
select date, AVG(impl_volatility) as iv  
from optionm.vsurfd%(year)s
where secid in %(secids)s and days = 30 and abs(delta)<=50
group by date
'''

params['year'] = 2000
data           = db.raw_sql(sql,params = params)

for y in range(2001,2021):
    print('Processing year ', y)
    params['year'] = y
    data = pd.concat([data, db.raw_sql(sql,params = params)])
    
data.loc[:,'date'] = pd.to_datetime(data.loc[:,'date'])

data = data.sort_values('date')

Processing year  2001
Processing year  2002
Processing year  2003
Processing year  2004
Processing year  2005
Processing year  2006
Processing year  2007
Processing year  2008
Processing year  2009
Processing year  2010
Processing year  2011
Processing year  2012
Processing year  2013
Processing year  2014
Processing year  2015
Processing year  2016
Processing year  2017
Processing year  2018
Processing year  2019
Processing year  2020
